In [1]:
pip install pyserial



  Obtaining dependency information for pyserial from https://files.pythonhosted.org/packages/07/bc/587a445451b253b285629263eb51c2d8e9bcea4fc97826266d186f96f558/pyserial-3.5-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB 1.3 MB/s eta 0:00:01
   ------------------------------------ --- 81.9/90.6 kB 907.3 kB/s eta 0:00:01
   ---------------------------------------- 90.6/90.6 kB 1.0 MB/s eta 0:00:00


# code with feature engineering

In [ ]:
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import time
import serial

# Establish connection to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="temperature_data"
)

cursor = db.cursor()

# Configure the external device
ser = serial.Serial('/dev/ttyUSB0', 9600)  # Adjust serial port and baud rate

# Function to fetch data from the external source
def fetch_external_data():
    line = ser.readline().decode('utf-8').strip()
    indoor_temp, outdoor_temp, desired_temp = map(float, line.split(','))
    return indoor_temp, outdoor_temp, desired_temp

# Function to insert data into the database
def insert_data(timestamp, indoor_temp, outdoor_temp, desired_temp):
    query = """
        INSERT INTO temperatures (timestamp, ntc_indoor_temperature, ntc_outdoor_temperature, desired_temperature)
        VALUES (%s, %s, %s, %s)
    """
    values = (timestamp, indoor_temp, outdoor_temp, desired_temp)
    cursor.execute(query, values)
    db.commit()

# Function to deleting the  data which is older than 3 minutes
def delete_old_data():
    query = "DELETE FROM temperatures WHERE timestamp < NOW() - INTERVAL 3 MINUTE"
    cursor.execute(query)
    db.commit()

# THis function fetches data from the database which is collected from sensor
def fetch_data():
    query = "SELECT timestamp, ntc_indoor_temperature, ntc_outdoor_temperature, desired_temperature FROM temperatures"
    cursor.execute(query)
    rows = cursor.fetchall()
    return pd.DataFrame(rows, columns=['timestamp', 'ntc_indoor_temperature', 'ntc_outdoor_temperature', 'desired_temperature'])

# Placeholder function for your ML model prediction
def predict_desired_temperature(indoor_temp, outdoor_temp, timestamp):
    return np.random.uniform(16, 24) #Random prediction between 16 and 24

# Adjusting the AC temperature 
def adjust_temperature(current_temp, user_preference, adjustment_rate=0.1):
    """
    Adjust the desired temperature towards the user preference.

    Parameters:
    - current_temp: Current desired temperature predicted by the model.
    - user_preference: User's preferred temperature.
    - adjustment_rate: Rate at which the temperature is adjusted.

    Returns:
    - new_temp: Adjusted desired temperature.
    """
    if current_temp < user_preference:
        new_temp = min(current_temp + adjustment_rate, user_preference)
    else:
        new_temp = max(current_temp - adjustment_rate, user_preference)

    return new_temp

# Example of user preference
user_preference = 22.0

# reading of the indoor and outdoor temperatures from the NTC sensors
def read_indoor_temperature():
    return np.random.normal(loc=27, scale=1)

def read_outdoor_temperature():
    return np.random.normal(loc=35, scale=1)

# In this loop to fetching data, inserting into the database, deleting data older tha 3 min, updating the model, and adjusting the temperature
while True:
    try:
        indoor_temp, outdoor_temp, desired_temp = fetch_external_data()
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        insert_data(timestamp, indoor_temp, outdoor_temp, desiredt_temp)
        delete_old_data()
        time.sleep(5)

        data = fetch_data()

        # Checking if data is sufficient to train the model
        if len(data) > 1:
            # Feature Engineering
            data['timestamp'] = pd.to_datetime(data['timestamp'])
            data['Hour'] = data['timestamp'].dt.hour
            data['Day'] = data['timestamp'].dt.day
            data['Month'] = data['timestamp'].dt.month

            # Select Features and Targe
            features = ['ntc_indoor_temperature', 'ntc_outdoor_temperature', 'Hour', 'Day', 'Month']
            X = data[features]
            y = data['desired_temperature']

            # Train-Test Split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Apply feature scaling to noemalize data
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Applying Decision Tree Regressor algorithm
            tree_model = DecisionTreeRegressor(random_state=42)
            tree_model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred_tree = tree_model.predict(X_test)

            # Printing evaluation metrics of the model to check performance
            mse_tree = mean_squared_error(y_test, y_pred_tree)
            r2_tree = r2_score(y_test, y_pred_tree)

            print(f'Decision Tree - Mean Squared Error: {mse_tree}')
            print(f'Decision Tree - R^2 Score: {r2_tree}')

            # Visualizing actual vs predicted values
            plt.scatter(X_test[:, 0], y_test, color='blue', label='Actual')
            plt.scatter(X_test[:, 0], y_pred_tree, color='green', label='Decision Tree Predicted')
            plt.xlabel('NTC Indoor Temperature (Scaled)')
            plt.ylabel('Desired Temperature')
            plt.legend()
            plt.show()

            #  reading temperatures and adjusting the desired temperature accordingly
            for _ in range(100):  #  100 time steps
                indoor_temp = read_indoor_temperature()
                outdoor_temp = read_outdoor_temperature()
                timestamp = datetime.now()

                # Predict the initial desired temperature using the ML model
                current_desired_temp = predict_desired_temperature(indoor_temp, outdoor_temp, timestamp)

                # Adjust the desired temperature based on user preference
                current_desired_temp = adjust_temperature(current_desired_temp, user_preference)

                print(f"Timestamp: {timestamp}, Indoor Temp: {indoor_temp:.2f}°C, Outdoor Temp: {outdoor_temp:.2f}°C, Adjusted Desired Temp: {current_desired_temp:.2f}°C")

                time.sleep(1)  # Sleep for a second and repeating the process again.
                # you can change time 

        else:
            print("Not enough data to train the model")
    except Exception as e:
        print(f"Error: {e}")
        break  # Exit the loop if there's an error
